## Generating constraint and violation data for building services on top of the project

In [1]:
from pathlib import Path
import os
from semconstmining.parsing.label_parser.nlp_helper import NlpHelper
from semconstmining.config import Config
from semconstmining.main import get_resource_handler, get_or_mine_constraints
from semconstmining.declare.enums import Template

MODEL_COLLECTION = "semantic_sap_sam_filtered"
config = Config(Path(os.getcwd()).parents[1].resolve(), MODEL_COLLECTION)
config.CONSTRAINT_TYPES_TO_IGNORE.remove(Template.NOT_CO_EXISTENCE.templ_str)

In [2]:
nlp_helper = NlpHelper(config)
resource_handler = get_resource_handler(config, nlp_helper)
all_constraints = get_or_mine_constraints(config, resource_handler, min_support=1)

[2023-08-02 14:18:34,704] p70703 {bert_wrapper.py:239} INFO - loading serialized model
[2023-08-02 14:18:36,443] p70703 {bert_wrapper.py:247} INFO - model loaded
[2023-08-02 14:18:37,341] p70703 {keyedvectors.py:2047} INFO - loading projection weights from /Users/adrianrebmann/gensim-data/glove-wiki-gigaword-50/glove-wiki-gigaword-50.gz
[2023-08-02 14:18:56,260] p70703 {utils.py:448} INFO - KeyedVectors lifecycle event {'msg': 'loaded (400000, 50) matrix of type float32 from /Users/adrianrebmann/gensim-data/glove-wiki-gigaword-50/glove-wiki-gigaword-50.gz', 'binary': False, 'encoding': 'utf8', 'datetime': '2023-08-02T14:18:56.260015', 'gensim': '4.3.1', 'python': '3.9.15 (main, Oct 11 2022, 22:27:25) \n[Clang 14.0.0 (clang-1400.0.29.102)]', 'platform': 'macOS-13.1-x86_64-i386-64bit', 'event': 'load_word2vec_format'}
[2023-08-02 14:18:56,363] p70703 {nlp_helper.py:96} INFO - Loaded 0 known embeddings and 238914 known similarities
[2023-08-02 14:18:56,364] p70703 {resource_handler.py:113

In [3]:
all_constraints

,obs_id,Level,Object,constraint_string,op_type,dictionary,data_object,template,left_op,right_op,model_id,model_name,support,redundant,nat_lang_template,ltl
1,d029cfdd-33ea-426e-b924-53e4482fd28e,Activity,,Exactly1[prepare order for shiping] | |,Unary,{},{},Exactly,prepare order for shiping,NaN,1bb03792264e4bd6914da6aafe094f34,BPMN,1,False,{1} occurs exactly {n} times,d029cfdd-33ea-426e-b924-53e4482fd28e := (F(pre...
2,0f72c47b-e29f-4c86-afc9-a425009b34c3,Activity,,Exactly1[record order] | |,Unary,{},{},Exactly,record order,NaN,1bb03792264e4bd6914da6aafe094f34 | ef5f51f1e97...,BPMN | Project Kickoff | tutorial bpmn 1,3,False,{1} occurs exactly {n} times,0f72c47b-e29f-4c86-afc9-a425009b34c3 := (F(rec...
3,66e26a81-f1e2-4adc-9676-0273fa133e45,Activity,,"Succession[record order, prepare order for shi...",Binary,{},{},Succession,record order,prepare order for shiping,1bb03792264e4bd6914da6aafe094f34,BPMN,1,False,{1} occurs if and only if it is followed by {2},66e26a81-f1e2-4adc-9676-0273fa133e45 := (G((re...
12,d050b805-0bb2-4135-aa70-491b56a3daa4,Activity,,"Alternate Succession[record order, prepare ord...",Binary,{},{},Alternate Succession,record order,prepare order for shiping,1bb03792264e4bd6914da6aafe094f34,BPMN,1,False,{1} and {2} occur if and only if the latter fo...,d050b805-0bb2-4135-aa70-491b56a3daa4 := (G((re...
14,f000fef5-1703-4387-a119-daa4d474f1fb,Activity,,Responded Existence[place prescription in labe...,Binary,{},{},Responded Existence,place prescription in labeled box,check fulfillment request,1bd1ef6b99d6492c9d533f9120462f18,Jane Doe - Receive prescription,1,False,"if {1} occurs in the process instance, then {2...",f000fef5-1703-4387-a119-daa4d474f1fb := (F(pla...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165697,fbb7654b-487e-4563-a546-af389b64ccaa,Multi-object,,"Succession[update min week, recommendation] | | |",Binary,{},{},Succession,update min week,recommendation,e8f3d14581d74d578b88f55cf9988ff3,task2_E_j04_a12345678,1,False,{1} occurs if and only if it is followed by {2},fbb7654b-487e-4563-a546-af389b64ccaa := (G((up...
165698,b941cd54-5e0d-4c34-8d4c-7ab8d1b34230,Multi-object,,Alternate Succession[field of interest money e...,Binary,{},{},Alternate Succession,field of interest money experience,recommendation,e8f3d14581d74d578b88f55cf9988ff3,task2_E_j04_a12345678,1,False,{1} and {2} occur if and only if the latter fo...,b941cd54-5e0d-4c34-8d4c-7ab8d1b34230 := (G((fi...
165699,6e66e451-00b1-42a2-af56-72cf621f87e7,Multi-object,,"Succession[field of interest money experience,...",Binary,{},{},Succession,field of interest money experience,update min week,e8f3d14581d74d578b88f55cf9988ff3,task2_E_j04_a12345678,1,False,{1} occurs if and only if it is followed by {2},6e66e451-00b1-42a2-af56-72cf621f87e7 := (G((fi...
165701,b38665ce-839e-4874-abf9-1e2a468ee22f,Multi-object,,"Alternate Succession[update min week, recommen...",Binary,{},{},Alternate Succession,update min week,recommendation,e8f3d14581d74d578b88f55cf9988ff3,task2_E_j04_a12345678,1,False,{1} and {2} occur if and only if the latter fo...,b38665ce-839e-4874-abf9-1e2a468ee22f := (G((up...


In [4]:
# Export
all_constraints.to_csv("example_constraints.csv")

### Creation violation data

In [5]:
from semconstmining.log.loghandler import LogHandler
from semconstmining.log.loginfo import LogInfo
from semconstmining.log.logstats import LogStats
from semconstmining.selection.instantiation.filter_config import FilterConfig
from semconstmining.selection.instantiation.recommendation_config import RecommendationConfig
from semconstmining.main import get_parts_of_constraints, compute_relevance_for_log, get_log_and_info, recommend_constraints_for_log, check_constraints, get_violation_to_cases
from semconstmining.selection.instantiation.constraintfilter import ConstraintFilter
from semconstmining.selection.instantiation.constraintrecommender import ConstraintRecommender
from os.path import exists
import pandas as pd


CURRENT_LOG_FILE = "BPI_Challenge_2019-3-w-after.xes"

filter_config = FilterConfig(config)
recommender_config = RecommendationConfig(config, top_k=250)

nlp_helper.pre_compute_embeddings(sentences=get_parts_of_constraints(config, all_constraints))

const_filter = ConstraintFilter(config, filter_config, resource_handler)
filtered_constraints = const_filter.filter_constraints(all_constraints)
event_log, log_info = get_log_and_info(config, nlp_helper, CURRENT_LOG_FILE)

# Log-specific constraint recommendation
if not exists(config.DATA_INTERIM / (CURRENT_LOG_FILE + "-constraints_with_relevance.pkl")):
    filtered_constraints = compute_relevance_for_log(config, filtered_constraints, nlp_helper, CURRENT_LOG_FILE,
                                                     pd_log=event_log, precompute=True)
    filtered_constraints.to_pickle(config.DATA_INTERIM / (CURRENT_LOG_FILE + "-constraints_with_relevance.pkl"))
else:
    filtered_constraints = pd.read_pickle(config.DATA_INTERIM / (CURRENT_LOG_FILE + "-constraints_with_relevance.pkl"))
recommended_constraints = recommend_constraints_for_log(config, recommender_config, filtered_constraints,
                                                        nlp_helper,
                                                        CURRENT_LOG_FILE, pd_log=event_log)
consistent_recommended_constraints = recommended_constraints
violations = check_constraints(config, CURRENT_LOG_FILE, consistent_recommended_constraints, nlp_helper, pd_log=event_log)
violations_to_cases = get_violation_to_cases(config, violations)
violation_df = pd.DataFrame.from_records(
    [{"violation": violation, "num_violations": len(cases), "cases": cases} for violation, cases in
     violations_to_cases.items()])

[2023-08-02 12:35:05,101] p68129 {SentenceTransformer.py:66} INFO - Load pretrained SentenceTransformer: /Users/adrianrebmann/Desktop/Best-Practice Conf./content/dev/data/all-MiniLM-L6-v2
[2023-08-02 12:35:05,562] p68129 {SentenceTransformer.py:105} INFO - Use pytorch device: cpu


Batches:   0%|          | 0/493 [00:00<?, ?it/s]

parsing log, completed traces ::   0%|          | 0/15182 [00:00<?, ?it/s]

1it [00:00, 24.91it/s]
1it [00:00, 25.45it/s]


Could not find a verb for complete


...

1it [00:00, 20.81it/s]
1it [00:00, 25.32it/s]
1it [00:00, 22.39it/s]
1it [00:00, 27.30it/s]
1it [00:00, 14.95it/s]
1it [00:00, 17.62it/s]
1it [00:00, 19.78it/s]
1it [00:00, 24.46it/s]
1it [00:00, 19.90it/s]
1it [00:00, 23.36it/s]
1it [00:00, 21.73it/s]
1it [00:00, 25.01it/s]


Could not find a verb for incomplete


1it [00:00, 25.70it/s]
1it [00:00, 29.78it/s]
1it [00:00, 21.49it/s]
1it [00:00, 23.38it/s]
1it [00:00, 19.56it/s]
1it [00:00, 20.18it/s]
1it [00:00, 23.97it/s]
1it [00:00, 19.50it/s]


Could not find a verb for record


1it [00:00, 19.07it/s]
1it [00:00, 27.51it/s]
1it [00:00, 29.91it/s]
1it [00:00, 26.13it/s]
1it [00:00, 25.00it/s]
[2023-08-02 12:35:57,853] p68129 {constraintrecommender.py:122} INFO - Computed relevance scores.
[2023-08-02 12:35:57,868] p68129 {constraintrecommender.py:124} INFO - Recommended 2577 constraints
[2023-08-02 12:36:01,304] p68129 {constraintfitter.py:27} INFO - Number of fitted constraints: 10749


In [6]:
violation_df

,violation,num_violations,cases
0,Exactly1[record] | |subsequent invoice,19,"[4507000449_00010, 4507000684_00010, 450700083..."
1,Exactly1[change] | |quantity,115,"[4507000322_00020, 4507000322_00030, 450700032..."
2,"Succession[set, check] | | |payment block",61,"[4507000430_00010, 4507000472_00001, 450700080..."
3,Absence1[set] | |payment block,61,"[4507000430_00010, 4507000472_00001, 450700080..."
4,"Alternate Succession[set, check] | | |payment ...",61,"[4507000430_00010, 4507000472_00001, 450700080..."
...,...,...,...
3172,"Succession[change payment term, notify staff t...",1,[4508060381_00010]
3173,"Alternate Succession[change payment term, noti...",1,[4508060381_00010]
3174,"Precedence[record goods receipt, clear invoice...",1,[4508071078_00010]
3175,"Responded Existence[clear invoice, vendor crea...",1,[4508071078_00010]


In [7]:
# Export
violation_df.to_pickle("example_violations.pkl")